# PreLoads

In [1]:
%run /workspaces/codespaces-jupyter/Workspace/References/PreLoads.ipynb

Install Packages

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

Import Packages

In [3]:
import os
import io
import openai
import requests
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

Load OpenAI API

In [4]:
OPENAI_API_KEY = dbutils.secrets.get(scope = "ahicks233", key = "OpenAI_API")

NameError: name 'dbutils' is not defined

# Main

Load PDF URL

In [ ]:
# URL of the PDF file
url = 'https://arxiv.org/pdf/2304.03442.pdf'

# Download the PDF file and read its contents
response = requests.get(url)
pdf_content = io.BytesIO(response.content)

# Do something with the PDF content, e.g. print the number of pages
import PyPDF2
reader = PyPDF2.PdfFileReader(pdf_content)
print(f"Number of pages: {reader.getNumPages()}")

Read PDF Data

In [5]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

raw_text[:100]

NameError: name 'reader' is not defined

Split Text Data into Blocks for Processing

In [ ]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

Load OpenAI API

In [6]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

NameError: name 'OPENAI_API_KEY' is not defined

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)
docsearch

In [ ]:
chain = load_qa_chain(OpenAI(openai_api_key=OPENAI_API_KEY), chain_type="stuff")

Q&A

In [ ]:
query = "Summarize this article?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

Q&A - FAILURE

In [ ]:
query = "Excluding the names in the reference section, who are all the authors of this article?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)